In [1]:
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Dropout
from keras_tqdm import TQDMNotebookCallback
import keras.callbacks

from sqlalchemy import create_engine
import json
import pickle

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
# R  1
# G  
# H  

category = 'R'
# category = 'G'
# category = 'H'

In [3]:
engine = create_engine('sqlite:///tab.db')
conn = engine.connect()
conn
dfoo = pd.read_sql_table('race', conn)
print('{} races loaded!'.format(len(dfoo)))

21699 races loaded!


In [4]:
dfo = dfoo.loc[dfoo['race_type'].isin([category])]
print('{} {} races!'.format(len(dfo), category))

7057 R races!


In [5]:
dfo['results'] = dfo['results_data'].map(json.loads)
dfo['runners'] = dfo['runners_data'].map(json.loads)
dfo.head()

/Users/jaco/.pyenv/versions/3.6.1/envs/tabby/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/jaco/.pyenv/versions/3.6.1/envs/tabby/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,id,meeting_name,location,venue_mnemonic,race_type,meeting_date,race_number,race_name,race_start_time,race_status,race_distance,results_data,num_runners,runners_data,quinella,exacta,trifecta,first_four,results,runners
1025,1610,RANDWICK,NSW,S,R,2017-09-16,1,TAB HIGHWAY HANDICAP,2017-09-16 02:15:00,Paying,1000,"[[4], [10], [2], [5]]",11,"[{""runnerName"": ""AZZIE'S READY"", ""runnerNumber...",605.4,NaN,NaN,NaN,"[[4], [10], [2], [5]]","[{'runnerName': 'AZZIE'S READY', 'runnerNumber..."
1026,1611,RANDWICK,NSW,S,R,2017-09-16,2,SHOOT OUT MILE,2017-09-16 02:50:00,Paying,1600,"[[3], [5], [8], [7]]",10,"[{""runnerName"": ""GOLD HORIZON"", ""runnerNumber""...",2296.9,NaN,NaN,NaN,"[[3], [5], [8], [7]]","[{'runnerName': 'GOLD HORIZON', 'runnerNumber'..."
1027,1612,RANDWICK,NSW,S,R,2017-09-16,3,SCHWEPPES HERITAGE STAKES,2017-09-16 03:25:00,Paying,1100,"[[2], [1], [8], [6]]",10,"[{""runnerName"": ""LUCKY LOUIE"", ""runnerNumber"":...",1421.1,NaN,NaN,NaN,"[[2], [1], [8], [6]]","[{'runnerName': 'LUCKY LOUIE', 'runnerNumber':..."
1028,1613,RANDWICK,NSW,S,R,2017-09-16,4,BILL RITCHIE HANDICAP,2017-09-16 04:00:00,Paying,1400,"[[7], [3], [4], [6]]",9,"[{""runnerName"": ""PAJARO"", ""runnerNumber"": 6, ""...",111.9,NaN,NaN,NaN,"[[7], [3], [4], [6]]","[{'runnerName': 'PAJARO', 'runnerNumber': 6, '..."
1029,1614,RANDWICK,NSW,S,R,2017-09-16,5,MCGRATH ESTATE TEA ROSE STAKES,2017-09-16 04:35:00,Paying,1400,"[[6], [2], [5], [1]]",12,"[{""runnerName"": ""PANDEMONIUM"", ""runnerNumber"":...",911.7,NaN,NaN,NaN,"[[6], [2], [5], [1]]","[{'runnerName': 'PANDEMONIUM', 'runnerNumber':..."


In [6]:
# extract runners from races
data_all = pd.DataFrame()
for ri, race in dfo.iterrows():
#     print(race)

    num_runners = len([r for r in race['runners'] if r['has_odds']])

    # add num_runners to all runners
    for runner in race['runners']:
#         print(runner)
        runner['num_runners'] = num_runners

    data_all = data_all.append(race['runners'])
    
data_all.tail(3)

,P_pred,P_prob,W_pred,W_prob,barrierNumber,claimAmount,finishingPosition,fixedOdds,has_odds,num_runners,...,place_rank,place_scaled,riderDriverName,runnerName,runnerNumber,trainerName,win_odds,win_perc,win_rank,win_scaled
10,0.349417,0.104326,0.130689,0.109959,9,-1.0,0,"{'returnWin': 5, 'returnWinOpen': 5, 'returnPl...",True,12,...,4.000000,0.093153,T EAVES,AUSPICION,4,T P TATE,5.0,0.200000,4.000000,0.107351
11,0.585536,0.174824,0.277830,0.233760,6,2.5,1,"{'returnWin': 2.2, 'returnWinOpen': 5, 'return...",True,12,...,1.000000,0.147492,C RODRIGUEZ,CORNBOROUGH,5,MARK WALFORD,2.2,0.454545,1.000000,0.243979
12,0.000000,0.000000,0.000000,0.000000,2,-1.0,0,"{'returnWin': 0, 'returnWinOpen': 0, 'returnPl...",False,12,...,2.564949,0.000000,A BESCHIZZA,CRAFTSMANSHIP,6,ROBERT EDDERY,0.0,0.000000,2.564949,0.000000


In [7]:
# drop scratched
data = data_all.dropna(subset=['win_odds', 'place_odds'])
data = data[(data.win_odds > 0)]
data = data[(data.place_odds > 0)]
data.describe()

,P_pred,P_prob,W_pred,W_prob,barrierNumber,claimAmount,finishingPosition,num_runners,place_odds,place_perc,place_rank,place_scaled,runnerNumber,win_odds,win_perc,win_rank,win_scaled
count,73153.000000,73153.000000,73153.000000,73153.000000,73153.000000,73153.000000,73153.000000,73153.00000,73153.000000,73153.000000,73153.000000,73153.000000,73153.000000,73153.000000,73153.000000,73153.000000,73153.000000
mean,0.254063,0.096469,0.092064,0.096469,5.943994,-0.346862,0.821538,11.33691,9.209378,0.348519,6.027709,0.096469,6.444411,21.435821,0.127011,6.056143,0.096469
std,0.185482,0.075277,0.091387,0.095161,3.623206,1.286466,1.282896,3.30614,98.259489,0.206349,3.719357,0.062304,3.957410,28.653048,0.130798,3.718357,0.093881
min,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,0.000000,4.00000,1.010000,0.000444,1.000000,0.000066,1.000000,1.010000,0.002494,1.000000,0.000768
25%,0.105626,0.038480,0.029032,0.030456,3.000000,-1.000000,0.000000,9.00000,2.050000,0.183486,3.000000,0.048167,3.000000,6.000000,0.043478,3.000000,0.031407
50%,0.219481,0.079731,0.063492,0.066582,5.000000,-1.000000,0.000000,11.00000,3.200000,0.312500,6.000000,0.083996,6.000000,12.000000,0.083333,6.000000,0.065740
75%,0.358613,0.135323,0.121363,0.128181,8.000000,0.000000,2.000000,13.00000,5.450000,0.487805,8.000000,0.133464,9.000000,23.000000,0.166667,9.000000,0.131033
max,0.998952,0.887873,1.000000,0.999854,24.000000,5.000000,4.000000,24.00000,2250.000000,0.990099,23.000000,0.441762,24.000000,401.000000,0.990099,23.000000,0.820518


In [8]:
# get label data
fp = data['finishingPosition']

yw = (fp == 1)
yp = (fp == 1) | (fp == 2) | ((fp == 3) & (data['num_runners'] >= 8))
Y = pd.concat([yw, yp], axis=1)

#print(data['finishingPosition'].head(10))
Y = Y.astype(int)
# Y.head(10)
Y.describe()

,finishingPosition,0
count,73153.000000,73153.000000
mean,0.096660,0.270570
std,0.295497,0.444257
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.000000,0.000000
75%,0.000000,1.000000
max,1.000000,1.000000


In [9]:
# xnr = get num runners
xnr = data['num_runners']
xnr.describe()

count    73153.00000
mean        11.33691
std          3.30614
min          4.00000
25%          9.00000
50%         11.00000
75%         13.00000
max         24.00000
Name: num_runners, dtype: float64

In [10]:
# xwr = win rank
xwr = data['win_rank']
# xfwr.describe()

# xwp = win perc
xwp = data['win_perc']
# xwp.describe()

# xws = win scaled
xws = data['win_scaled']
# xws.describe()

In [11]:
# xpr = place rank
xpr = data['place_rank']
# xpr.describe()

# xpp = place perc
xpp = data['place_perc']
# xpp.describe()

# xps = place scaled
xps = data['place_scaled']
# xps.describe()

In [12]:
X = pd.concat([xnr, 
               xwr, xwp, xws,
               xpr, xpp, xps], axis=1)
X.describe()
# sns.pairplot(pd.concat([X, Y], axis=1))

,num_runners,win_rank,win_perc,win_scaled,place_rank,place_perc,place_scaled
count,73153.00000,73153.000000,73153.000000,73153.000000,73153.000000,73153.000000,73153.000000
mean,11.33691,6.056143,0.127011,0.096469,6.027709,0.348519,0.096469
std,3.30614,3.718357,0.130798,0.093881,3.719357,0.206349,0.062304
min,4.00000,1.000000,0.002494,0.000768,1.000000,0.000444,0.000066
25%,9.00000,3.000000,0.043478,0.031407,3.000000,0.183486,0.048167
50%,11.00000,6.000000,0.083333,0.065740,6.000000,0.312500,0.083996
75%,13.00000,9.000000,0.166667,0.131033,8.000000,0.487805,0.133464
max,24.00000,23.000000,0.990099,0.820518,23.000000,0.990099,0.441762


In [14]:
# For a single-input model with 2 classes (binary classification):

n = len(X.columns)
print('input dimension = {}'.format(n))

epochs = 250
print('epochs = {}'.format(epochs))

tag = '{}50x50'.format(category)
print('tag = {}'.format(tag))
file_name = '/Users/jaco/code/tabby/models/{}.h5'.format(tag)

try:
    model = load_model(file_name)
except OSError:
    model = Sequential()
    model.add(Dense(50, activation='relu', input_dim=n))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(2, activation='sigmoid'))
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

input dimension = 7
epochs = 250
tag = R50x50


In [15]:
# Train the model, iterating on the data in batches of 32 samples
tqdm = TQDMNotebookCallback()

tbCallBack = keras.callbacks.TensorBoard(
    log_dir='/Users/jaco/code/tabby/summary/{}'.format(tag), 
    histogram_freq=0,
    write_graph=True,
    write_images=True)

model.fit(
    X.as_matrix(), 
    Y.as_matrix(),
    validation_split=0.2,
    shuffle=True,
    epochs=epochs,
    batch_size=32,
    verbose=0,
    callbacks=[tqdm, tbCallBack])

# creates a HDF5 file 'my_model.h5'
model.save(file_name)